## Task 3: Call Bedrock models with LangChain

In this task, you will create tools to sell more tickets and book shows. In the process of asking Bedrock LLMs questions and assigning them tasks to run booking, you should compare the code needed to invoke models through an AWS SDK (Boto3) and LangChain.

1. Choose **Select Kernel** in the top right corner and then in the **Select kernel** dialog that appears select **Install/Enable suggested extensions Python + Jupyter**.

    **Note:** If you see a popup window, choose **Trust Publisher & Install**.

2. Choose **Select Kernel** again, then in the **Select kernel** dialog choose **Python environments**. Then choose **Create Python Environment** and choose **Venv**, then choose **Python 3.12.11 64-bit**.

3. To install required the Python libraries for this lab, select **requirements.txt**, then press **OK**.

    **Note:** LangChain has multiple different packages for different purposes.
    In this lab, you use:
    - langchain: For its output parsers
    - langchain-community: For its document loaders
    - langchain_aws: For its AWS models
    - langchain_core: For its prompt templates and output parsers
    
    A few notifications appear, indicating that pip is being upgraded and other packages are being installed.

    Within a short time, where the notebook previously displayed the "Select Kernel" message, it should now display ".venv (Python 3.12.11)".

4. To suppress unnecessary warnings, run the following cell.

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

**Note:** When you place your cursor in a code cell, a **play** icon will appear on the left side. Use that to run the code. You will know the code block has completed running when you see a number display within square brackets below the play icon.

### Task 3.1: Invoke models using boto3

Use *AWS SDK for Python (Boto3)* to ask an Amazon Nova Lite model how to get more people in the door.

5. To help the venue think of ideas to sell more tickets, perform a text completion task with *Amazon Nova Lite* through Boto3.

In [2]:
import boto3
import json
from IPython.display import Markdown


# Create the Amazon Bedrock client
bedrock_client = boto3.client('bedrock-runtime')

# Define the text generation configuration
textGenerationConfig = {
    "maxTokens": 512,
    "temperature": 0.5,
    "topP": 0.9
}

modelId = "amazon.nova-lite-v1:0"

# Define the input text for text generation
prompt = "A music venue can sell out every night by..."

native_request = {
    "messages": [
        {
            "role": "user",
            "content": [{"text": prompt}]
        }
    ],
    "inferenceConfig": textGenerationConfig
}

request = json.dumps(native_request).encode('utf-8')

# Invoke the Amazon Bedrock model for text generation
response = bedrock_client.invoke_model(modelId=modelId, body=request)

# Extract the outputText from the response
response = json.loads(response["body"].read())
generated_text = response.get('output').get('message').get('content')[0].get('text')

# Print the generated text
print(generated_text)

A music venue can sell out every night by employing a combination of strategic planning, marketing, and community engagement. Here are some key factors that contribute to consistent sell-outs:

1. **Diverse Lineup**:
   - **Variety of Genres**: Offer a mix of genres to attract a broad audience. This could include rock, pop, indie, electronic, jazz, and more.
   - **Local and National Acts**: Feature both local talent and well-known national or international artists to attract different segments of the audience.

2. **Consistent Quality**:
   - **Reputation**: Build a reputation for high-quality performances and a great venue experience. Word-of-mouth and online reviews play a significant role.
   - **Professionalism**: Ensure that every aspect of the event, from sound and lighting to security and staff, is top-notch.

3. **Engaging Marketing**:
   - **Social Media**: Use platforms like Instagram, Facebook, and Twitter to promote events, share behind-the-scenes content, and engage with 

Model invocation with Boto3 can be seen as low-level. If you want to make a complex, production-ready LLM tool, it requires you to do lots of things yourself in Python.

See the length of code required to invoke above and the return object that requires indexing to print pretty as some examples.

### Task 3.2: Invoke models using ChatBedrock with simple prompts

Leveraging LangChain components and abstractions to invoke Amazon Bedrock models can make your code more high-level. LangChain handles the low-level API details, response parsing, and error handling, allowing you to focus on building your application logic rather than managing model integration complexities.

You will use Amazon Nova Lite through LangChain's ChatBedrock component, which handles both single prompts and multi-turn conversations effectively.

6. Create a LangChain ChatBedrock component that uses *Amazon Nova Lite*.

In [3]:
# Import the ChatBedrock class from the langchain_aws.chat_models.bedrock module
from langchain_aws.chat_models.bedrock import ChatBedrock

# Set the modelId to the desired model ID (in this case, "amazon.nova-lite-v1:0")
modelId = "amazon.nova-lite-v1:0"

# Create an instance of the ChatBedrock with the specified model ID
nova_llm = ChatBedrock(model_id=modelId)

7. To brainstorm bands to reach out to, ask the LLM what the top selling bands of all time are.

In [4]:
# Define the prompt to be sent to the LLM
prompt = "What are the top-selling bands of all time?"

# Call the invoke method of the ChatBedrock instance with the prompt formatted as a human message and store the response
response = nova_llm.invoke([("human", prompt)])

# Render the response content as Markdown using the Markdown function
Markdown(response.content)

Determining the "top-selling" bands of all time can be somewhat subjective, as it often depends on the source and criteria used for measurement. However, several bands are widely recognized as the best-selling music artists globally based on their recorded music sales. Here are some of the top-selling bands of all time:

1. **The Beatles**  
   - Estimated sales: Over 600 million records worldwide.
   - The Beatles are often cited as the best-selling band of all time, with a massive global impact on music and culture.

2. **Led Zeppelin**  
   - Estimated sales: Over 300 million records worldwide.
   - Known for their powerful rock music, Led Zeppelin has sold millions of albums worldwide.

3. **Pink Floyd**  
   - Estimated sales: Over 250 million records worldwide.
   - Famous for their concept albums and progressive rock style, Pink Floyd has a dedicated global fanbase.

4. **Eagles**  
   - Estimated sales: Over 200 million records worldwide.
   - The Eagles are known for their blend of rock and country, with numerous hit songs and albums.

5. **Fleetwood Mac**  
   - Estimated sales: Over 100 million records worldwide.
   - Known for their hit songs and strong female presence, Fleetwood Mac has enjoyed long-lasting success.

6. **AC/DC**  
   - Estimated sales: Over 200 million records worldwide.
   - Renowned for their hard rock and heavy metal contributions, AC/DC has a massive following.

7. **The Rolling Stones**  
   - Estimated sales: Over 240 million records worldwide.
   - One of the most influential rock bands, the Rolling Stones have had a prolific career spanning several decades.

8. **Metallica**  
   - Estimated sales: Over 200 million records worldwide.
   - As leaders of the thrash metal genre, Metallica has sold millions of albums worldwide.

9. **U2**  
   - Estimated sales: Over 170 million records worldwide.
   - Known for their anthemic rock songs and political activism, U2 has a strong global presence.

10. **Bon Jovi**  
    - Estimated sales: Over 100 million records worldwide.
    - With a string of hit songs and albums, Bon Jovi has maintained a loyal fanbase for decades.

These figures are approximations and can vary based on different sources and methodologies. Nonetheless, these bands are universally acknowledged for their significant contributions to the music industry and their massive sales figures.

Maybe this independent venue should be realistic about their booking potential.

8. Prompt the LLM again, this time, with a caveat.

In [5]:
prompt = "What are 10 bands right now that only play small venues? Describe each band in detail."

# Call the invoke method of the ChatBedrock instance with the prompt formatted as a human message and store the response
response = nova_llm.invoke([("human", prompt)])

# Render the response content as Markdown using the Markdown function
Markdown(response.content)


Sure, here are ten bands that currently play primarily in small venues, along with detailed descriptions of each:

1. **The Districts**
   - **Genre**: Indie Rock/Alternative
   - **Origin**: Los Angeles, California
   - **Description**: The Districts, fronted by singer-songwriter Jacob Slater, emerged in the early 2010s. Their music is characterized by its raw, emotional intensity and introspective lyrics. Their sound blends elements of indie rock and alternative, often featuring Slater's haunting vocals and intricate guitar work. They have a dedicated following, often performing in intimate venues that allow for a close connection with their audience.

2. **The Head and the Heart**
   - **Genre**: Indie Folk/Rock
   - **Origin**: Seattle, Washington
   - **Description**: The Head and the Heart formed in 2010 and quickly gained a reputation for their heartfelt, anthemic sound. Their music combines indie folk and rock, with lush harmonies and poetic lyrics. The band's live performances are known for their energy and emotional depth, often filling small venues with their infectious sound.

3. **The War on Drugs**
   - **Genre**: Indie Rock/Psychedelic
   - **Origin**: Philadelphia, Pennsylvania
   - **Description**: Led by songwriter and multi-instrumentalist Adam Granduciel, The War on Drugs creates a dreamy, psychedelic sound that blends indie rock with elements of soul and funk. Their music is often characterized by its lush instrumentation and introspective lyrics. Despite their growing popularity, they still frequently play smaller venues, where their intricate soundscapes can be fully appreciated.

4. **The Ballroom Thieves**
   - **Genre**: Indie Rock/Folk
   - **Origin**: Los Angeles, California
   - **Description**: The Ballroom Thieves, fronted by singer-songwriter Patrick Stickles, combine elements of indie rock and folk. Their music is characterized by its poetic lyrics and intricate guitar work. The band often performs in small venues, where their storytelling and musical craftsmanship can resonate with audiences on a personal level.

5. **The Headstones**
   - **Genre**: Alternative Rock/Punk
   - **Origin**: Toronto, Ontario, Canada
   - **Description**: The Headstones are a Canadian alternative rock band known for their energetic live performances and socially conscious lyrics. Their music blends elements of punk, rock, and folk, often featuring powerful vocals and driving rhythms. They have a loyal following and frequently play in smaller venues, where their high-energy shows can engage the audience directly.

6. **The District Sleeps Alone Tonight**
   - **Genre**: Indie Rock/Alternative
   - **Origin**: Brooklyn, New York
   - **Description**: This Brooklyn-based band, led by singer-songwriter Joe Hertler, creates a mix of indie rock and alternative sounds. Their music is characterized by its introspective lyrics and intricate instrumentation. They often perform in small venues, where their detailed, atmospheric sound can create an intimate connection with the audience.

7. **The Head and the Heart**
   - **Genre**: Indie Folk/Rock
   - **Origin**: Seattle, Washington
   - **Description**: (Note: This is a repeat, but worth mentioning again for their intimate performances) The Head and the Heart formed in 2010 and quickly gained a reputation for their heartfelt, anthemic sound. Their music combines indie folk and rock, with lush harmonies and poetic lyrics. The band's live performances are known for their energy and emotional depth, often filling small venues with their infectious sound.

8. **The Districts**
   - **Genre**: Indie Rock/Alternative
   - **Origin**: Los Angeles, California
   - **Description**: (Note: This is a repeat, but worth mentioning again for their raw, emotional performances) The Districts, fronted by singer-songwriter Jacob Slater, emerged in the early 2010s. Their music is characterized by its raw, emotional intensity and introspective lyrics. Their sound blends elements of indie rock and alternative, often featuring Slater's haunting vocals and intricate guitar work. They have a dedicated following, often performing in intimate venues that allow for a close connection with their audience.

9. **The Head and the Heart**
   - **Genre**: Indie Folk/Rock
   - **Origin**: Seattle, Washington
   - **Description**: (Note: This is a repeat, but worth mentioning again for their heartfelt, anthemic performances) The Head and the Heart formed in 2010 and quickly gained a reputation for their heartfelt, anthemic sound. Their music combines indie folk and rock, with lush harmonies and poetic lyrics. The band's live performances are known for their energy and emotional depth, often filling small venues with their infectious sound.

10. **The Ballroom Thieves**
    - **Genre**: Indie Rock/Folk
    - **Origin**: Los Angeles, California
    - **Description**: (Note: This is a repeat, but worth mentioning again for their poetic, intricate performances) The Ballroom Thieves, fronted by singer-songwriter Patrick Stickles, combine elements of indie rock and folk. Their music is characterized by its poetic lyrics and intricate guitar work. The band often performs in small venues, where their storytelling and musical craftsmanship can resonate with audiences on a personal level.

These bands are known for their unique sounds and intimate performances, making them a great fit for smaller venues where their music can truly shine.

### Task 3.3: Invoke models using ChatBedrock with structured messages

You can also use LangChain to work with Bedrock models in a more conversational way with structured messages. LangChain chat models use a sequence of messages as inputs and return messages (as opposed to plain text) as outputs. You will use LangChain's chat model component, ChatBedrock, to book shows.

When working with a chat model, each message contains a *role* and *content*. In this lab, you work with the following:
- Human messages: Represents a message from a user
- AI messages: Represents a message from a model
- System messages: Represents a system messages, which tells the model how to behave

11. Import the ChatBedrock class, create an inference request parameters object, and define the Bedrock model id.

In [6]:
# Import the ChatBedrock class from the langchain_aws.chat_models.bedrock module
from langchain_aws.chat_models.bedrock import ChatBedrock

# Define the text generation configuration
textGenerationConfig = {
    "maxTokens": 512,
    "temperature": 0.5,
    "topP": 0.9
}

modelId = "amazon.nova-lite-v1:0"


12. **Challenge:** Create a **ChatBedrock** object, assigning it to a variable called **chat**, using **textGenerationConfig** and **modelId** as the inputs to the necessary parameters
[LangChain API Reference](https://python.langchain.com/v0.2/api_reference/aws/chat_models/langchain_aws.chat_models.bedrock.ChatBedrock.html).

In [7]:
# Create an Amazon Nova Lite LangChain chat model

nova_chat = ChatBedrock(
    client=bedrock_client,
    model_id=modelId,
    model_kwargs=textGenerationConfig,
)

<details>
    <summary><b>Solution:</b> Select if you need to see the solution.</summary>
    
<br/>

```python
# Create an Amazon Nova chat model object
nova_chat = ChatBedrock(
    client=bedrock_client,
    model_id=modelId,
    model_kwargs=textGenerationConfig,
)

```
</details>

LangChain chat models work with lists of messages, each with a persona and content.

13. Because you are working with a chat model, create a messages list with a *system message*, describing the model's purpose to fulfill, and a *content*, the question or response.

In [8]:
# Create a messages list to join in on the middle of a conversation with the chat model
messages = [
    (
        "system",
        "You are the manager of a music venue. You respond to artists who reach out to you about playing a show at your venue on their upcoming tour."
    ),
    (
        "human", 
        "Hello! We are an up-and-coming punk band with thousands of fans. We are coming to town September 17"
    )
]

14. Invoke the chat model, using the messages list as a parameter.

In [9]:
# Unlike, an LLM object, LangChain chat models, take a list of messages as a parameter for invocation.
ai_msg = nova_chat.invoke(messages)

**Learn more:** See [LangChain ChatBedrock documentation](https://python.langchain.com/v0.2/api_reference/aws/chat_models/langchain_aws.chat_models.bedrock.ChatBedrock.html) for a list of methods and ways to invoke this model.

The AI response comes in *Message* format. To avoid printing metadata and persona, index to the *content* key.

15. Print the content of the AI response.

In [10]:
Markdown(ai_msg.content)

Hello! Thank you for reaching out about playing a show at our venue on September 17. We're excited to hear about your upcoming tour and the buzz around your music.

To move forward, we'll need a bit more information to ensure we can accommodate your needs and provide the best experience for your fans. Here are a few details we'll need from you:

1. **Show Details:**
   - Preferred show time (start and end times).
   - Any special requests or requirements for the performance (e.g., stage setup, lighting, sound).

2. **Contract and Financials:**
   - Proposed show fee or revenue-sharing model.
   - Any additional costs (e.g., travel, accommodation, marketing support).

3. **Marketing and Promotion:**
   - How you plan to promote the show.
   - Any promotional materials or support you need from our end (e.g., posters, social media promotion).

4. **Logistics:**
   - Rider details (technical requirements, backstage needs, etc.).
   - Any specific logistical considerations (e.g., entry/exit points, parking).

5. **Support Acts:**
   - If you have any support acts or opening bands, please provide their details as well.

Once we have this information, we can discuss the details further and work towards a mutually beneficial agreement. We look forward to potentially hosting your show and seeing the energy you bring to our venue.

Feel free to reach out with any questions or additional information. We're excited to collaborate!

Best regards,

[Your Name]  
[Your Position]  
[Venue Name]  
[Contact Information]  
[Venue Website]

### Task 3.4: Challenge: Calculate cost of invocation

Use LangChain methods and Bedrock documentation to calculate the cost of your last LLM invocation.

When preparing to launch LLM applications, it is important to have an understanding of cost. If used by large organizations or repeatedly, Amazon Bedrock usage costs can add up.

1.  Find a method in the [LangChain documentation](https://api.python.langchain.com/en/latest/chat_models/langchain_aws.chat_models.bedrock.ChatBedrock.html) that returns the number of tokens present in a text string.

1.  Use that method on **prompt** and assign the output to a variable called **input_tokens**.


In [11]:
# Assign the number of tokens used in prompt to a variable called input_tokens
input_tokens = nova_chat.get_num_tokens(prompt)

/home/ec2-user/environment/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


18. Use the same method on **ai_message.content** and assign the output to a variable called **output_tokens**.

In [12]:
# Assign the number of tokens used in ai_message.content to a variable called output_tokens
output_tokens = nova_chat.get_num_tokens(ai_msg.content)


<details>
    <summary><b>Solution:</b> Select if you need to see the solution. If a warning message appears below the cell after you run it, you can safely ignore the message or run the cell again. </summary>
    
<br/>

```python
# The number of tokens the prompt uses for Amazon Nova Lite
input_tokens = nova_chat.get_num_tokens(prompt)
# The number of tokens the AI response used for Amazon Nova Lite
output_tokens = nova_chat.get_num_tokens(ai_msg.content)
```
</details>


19. Scan through [Amazon Bedrock pricing](https://aws.amazon.com/bedrock/pricing/) to find (and assign) the following values for **Amazon Nova Lite** (On-Demand, US East):

- **Price per 1,000 input tokens**, assigning that rate to variable **input_price** (price/1000)
- **Price per 1,000 output tokens**, assigning that rate to variable **output_price** (price/1000)

In [13]:
# Create two new variables for respective token prices

input_price = .0002/1000

<details>
    <summary><b>Solution:</b>Select if you need to see the solution.</summary>
    
<br/>

```python
# The listed price per 1,000 input tokens for using Amazon Nova Lite, divided by 1,000, to get the price of a single token
input_price = .0002/1000
# The price per 1,000 output tokens for using Amazon Nova Lite, divided by 1,000, to get the price of a single token
output_price = .0006/1000
```
</details>

20. Create an equation to calculate the cost of the model invocation in dollars and print the result.

In [33]:
# Print the cost of the model invocation (input and output)
output_price = .0006/1000



<details>
    <summary><b>Solution:</b> Select if you need to see the solution.</summary>
    
<br/>

```python
# Cost = quantity * price
cost_cents = input_tokens * input_price + output_tokens * output_price
# Multiply by 100 to go from cents to dollars
cost_dollars = cost_cents * 100
(f"The price is ${cost_dollars}")
```
</details>

While, the cost of this one invocation is very small, it's important to have an understanding of such costs. Such an invocation, compounded thousands or millions of times for a large organization or programmatic task, could have a massive cost.

**Task complete**: You built tools for the venue to sell more tickets and book shows using LangChain models and their methods.

---

## Task 4: Explore LangChain class capabilities

In this task, you will create parts of AI tools to help make the venue bar manager's job easier.
You'll use some new basic tools of LangChain, Messages, Prompt Templates, and Parsers, that abstract low level code to do common tasks when working to productionalize AI applications.

### Task 4.1: Messages

In the last task you worked with messages as a list of tuples. To simplify working with chat models, LangChain Core offers messages classes. Use messages to communicate with a chat model.

Generally, messages passed to a chat model should start with a *SystemMessage*. A SystemMessage can the general task and guidelines a chat model should adhere to.

21. Create a list of message objects:
- Priming the model with **SystemMessage**.
- Conveying the first message from a user with **HumanMessage**.

In [34]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are an AI assistant helping a music venue bar with administrative tasks."),
    HumanMessage(content="Draft an email to all bar staff reminding them of the updated closing procedures and checklists that need to be completed each night after events."),
]

22. Invoke the chat model.

In [35]:
response = nova_chat.invoke(messages)
print(response.content)

Subject: Important: Updated Closing Procedures and Checklists

Dear Bar Staff,

I hope this message finds you well.

As part of our ongoing efforts to ensure smooth and efficient operations, we have updated our closing procedures and checklists. It is crucial that all staff members adhere to these updated protocols to maintain the high standards of our venue.

**Effective immediately, please follow the new procedures and checklists outlined below:**

### Updated Closing Procedures:
1. **Event Wrap-Up:**
   - Ensure all guests have left the premises.
   - Confirm with security that the area is clear and secure.

2. **Bar Area:**
   - Clean all bar surfaces and equipment.
   - Replenish supplies and stock for the next event.
   - Ensure all glassware and utensils are washed and stored properly.

3. **Dining Area:**
   - Clear and reset all tables and chairs.
   - Clean all dining surfaces and equipment.
   - Ensure all dining ware is washed and stored.

4. **Restrooms:**
   - Clean all f

You can also stream chunks of the model response for a more pleasant, faster user experience.

23. Invoke the model again, streaming its response.

In [36]:
# Iterate over the stream of responses from the AI model
for chunk in nova_chat.stream(prompt):
    # Extract and print the text content from Nova's structured response
    if hasattr(chunk, 'content') and chunk.content:
        if isinstance(chunk.content, list):
            for item in chunk.content:
                if isinstance(item, dict) and item.get('type') == 'text':
                    # Print the content of each response chunk without adding a newline character
                    # end="" prevents printing a newline after each chunk
                    # flush=True ensures the output is flushed immediately, without buffering
                    print(item.get('text', ''), end="", flush=True)
        else:
            print(chunk.content, end="", flush=True)


pomegranate juice, cranberry juice, cherry juice, raspberry syrup, strawberry syrup, unsweetened cranberry juice, unsweetened pomegranate juice, unsweetened berry juice blend

### Task 4.2: Prompt templates

LangChain Prompt templates allow you to create reusable prompts with placeholders that can be filled with specific inputs, separating the structure of the prompt from the data. Use prompt templates to create purchase orders and come up with new drinks to put on the menu.

A **PromptTemplate** can be used with llm objects to inject inputs into a standard prompt design, creating prompts as strings.

24. To use a prompt template to create several prompts dynamically, run the following code.

In [18]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template("What is the capital of {state}?")

prompt = prompt_template.format(state = "Nebraska")
print("Prompt 1: ", prompt)

prompt = prompt_template.format(state = "New York")
print("Prompt 2: ", prompt)

Prompt 1:  What is the capital of Nebraska?
Prompt 2:  What is the capital of New York?


Now, lets create a tool for the bar manager using prompt templates.

25. Create a list of orders the bar needs to make.

In [37]:
orders = [
    {
        "product" : "soda",
        "supplier" : "TheSodaCompany, LLC",
        "date" : "9/10/2024"
    },
    {
        "product" : "napkins",
        "supplier" : "Napkin Inc.",
        "date" : "9/12/2024"
    },
    {
        "product" : "receipt paper",
        "supplier" : "Paper Unlimited",
        "date" : "9/19/2024"
    }

]


You will use each dictionary in the orders list to generate prompts, substituting the placeholders {product}, {supplier}, and {date} with the corresponding values from each dictionary.

In the following template, there are placeholders (i.e. the product and supplier and date) that match with keys in the dictionaries inside the order list.

26. Create a reusable prompt template that makes purchase orders to the venues supplier.

In [38]:
template="""
Human: Create a purchase order for {product} to {supplier} from our company, AMusicVenue,
stating that we require delivery by {date}.

Assistant:"""


Variables, in {}, allow prompts to be reused by replacing the placeholders with real values.

27. Turn the string into a PromptTemplate object, using the *from_template()* method.

In [39]:
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['date', 'product', 'supplier'], input_types={}, partial_variables={}, template='\nHuman: Create a purchase order for {product} to {supplier} from our company, AMusicVenue,\nstating that we require delivery by {date}.\n\nAssistant:')

A prompt template allows you to parameterize prompts, making them reusable. To turn the prompt template into a prompt, you assign values into the variables within the template using *format()* or *invoke()*.

28. To create a prompt, use that prompt to invoke a Bedrock model, and print the response, for each of your three needed product orders, run the following code.

In [22]:
nova_llm = ChatBedrock(model_id="amazon.nova-lite-v1:0",
                         model_kwargs={
                             "temperature": .8,
                             "topP": .8
                         })

# PO Counter
order_num = 1

# Loop through the orders list
for order in orders:
    # Create a prompt from a template by indexing the order dict
    prompt = prompt_template.format(product=order["product"],
                             supplier=order["supplier"],
                             date=order["date"])
    # Invoke a Bedrock model
    response = nova_llm.invoke([("human", prompt)])

    # Print the order #
    print("PO #" + str(order_num) + "\n")
    # Increment the order # counter
    order_num += 1
    # Print the model response
    print(response.content)
    print ("---------------------")


PO #1

**Purchase Order**

**To:**  
TheSodaCompany, LLC  
[Address of TheSodaCompany, LLC]  
[City, State, ZIP Code]  

**From:**  
AMusicVenue  
[Address of AMusicVenue]  
[City, State, ZIP Code]  

**Date:**  
[Current Date]

**Purchase Order Number:**  
[PO Number]

**Attention:**  
[Contact Person's Name, if known]

**Subject:**  
Purchase Order for Soda Delivery

---

**Order Details:**

**Product Description:**  
- Quantity: [Specify Quantity]  
- Product: [Specify Type of Soda, e.g., Coca-Cola, Pepsi, Sprite, etc.]  
- Packaging: [Specify Packaging, e.g., 12-pack, 24-pack, etc.]

**Total Amount:**  
- [Specify Total Amount in Words]  
- [Specify Total Amount in Numbers]

**Delivery Details:**  
- Delivery Address: [Specify Delivery Address if different from billing address]  
- Delivery Date: 9/10/2024

**Payment Terms:**  
- [Specify Payment Terms, e.g., Net 30, Net 60, etc.]

**Additional Notes:**  
- Please ensure that all products are in good condition and meet our quality 

**Note:** This code generated purchase order prompts by substituting placeholders in the template with values from a list of orders. It sent the prompts to an Amazon Bedrock LLM and printed the model's responses along with the purchase order numbers.

There are the bars purchase orders to its vendors for the week!

The **ChatPromptTemplate** is a tool that allows you to define a reusable template for conversational prompts, consisting of multiple messages with designated roles (e.g., system, user, assistant). It provides a structured way to incorporate user inputs or placeholders into the message list, which can then be used with LangChain chat models.

29. To create a chat prompt template to help the bar manager when they have to make a drink they don't know, create the following template.

In [23]:
# Import the ChatPromptTemplate class from the langchain_core.prompts module
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate(
  [
    # Define the initial system message for the chat prompt
    ("system", "You are a bar-keeper's assistant. When a user says a drink name, you respond briefly, giving them only the ingredients"),
    # Define the user's input message for the chat prompt
    # The {drink} placeholder will be substituted with the actual drink name provided by the user
    ("user", "{drink}"),
   ]
)


30. Create a prompt from the prompt template, which will prompt the chat model to list the ingredients for a delicious drink.

In [24]:
# Create a formatted prompt by substituting "shirley temple" for the {drink} placeholder in the chat_template
prompt = chat_template.invoke("shirley temple")

31. Stream the response from the chat model.

In [40]:
# Iterate over the stream of responses from the AI model
for chunk in nova_chat.stream(prompt):
    # Extract and print the text content from Nova's structured response
    if hasattr(chunk, 'content') and chunk.content:
        if isinstance(chunk.content, list):
            for item in chunk.content:
                if isinstance(item, dict) and item.get('type') == 'text':
                    # Print the content of each response chunk without adding a newline character
                    # end="" prevents printing a newline after each chunk
                    # flush=True ensures the output is flushed immediately, without buffering
                    print(item.get('text', ''), end="", flush=True)
        else:
            print(chunk.content, end="", flush=True)


pomegranate juice, cranberry juice, cherry juice, raspberry syrup, strawberry syrup, unsweetened cranberry juice, unsweetened pomegranate juice, unsweetened berry juice blend

### Task 4.3: Output parsers

Language models typically produce unstructured text outputs. When your application requires structured data, in a specific format, you need to prompt LLMs for very specific formats, which they might be resistant to, or transform outputs to meet your requirements. LangChain output parsers can be used to transform the output of an LLM to a more suitable format. 

Use output parsers to get outputs of specific formats from LLMs.

A **CommaSeperatedListOutputParser** component provides several abilities. You can:
- Add its format instructions to a prompt template, which will inform the chat model on what to return. Doing such will add the following to your prompt: "Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`".
- Invoke it to parse the model response from a string to a list of strings.

When a bartender runs out of an ingredient on a show night, they might have to find another way to make a Shirley Temple on the fly.

32. Create a prompt template to ask the chat model for a commas separated list of potential replacements for an ingredient.

In [41]:
# Import the CommaSeparatedListOutputParser class from the langchain.output_parsers module
from langchain.output_parsers import CommaSeparatedListOutputParser

# Create an instance of the CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()

# Get the format instructions from the output parser
# This will return a string with instructions for the language model
# to format its output as a comma-separated list
format_instructions = output_parser.get_format_instructions()

# Create a PromptTemplate object
# The template string includes a placeholder for the format instructions
# and a placeholder for the input variable "ingredient"
prompt_template = PromptTemplate(
    template="List substitutes for {ingredient}.\n{format_instructions}",
    input_variables=["ingredient"],
    partial_variables={"format_instructions": format_instructions},
)

33. Create a prompt from the prompt template.

In [27]:
# Use the PromptTemplate object to create a prompt for the language model
# by invoking it with the input variable "ingredient" set to "grenadine"
prompt = prompt_template.invoke({"ingredient": "grenadine"})

# Print the generated prompt string
print(prompt.to_string())

List substitutes for grenadine.
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


34. Observe the model response before CommaSeparatedListOutputParser is invoked.

In [28]:
nova_chat = ChatBedrock(model_id="amazon.nova-lite-v1:0", model_kwargs = {
    "temperature": .2,
    "top_p": .2})

response = nova_chat.invoke(prompt)
print(response.content)

pomegranate juice, cranberry juice, cherry juice, raspberry juice, strawberry syrup, elderflower cordial, orange juice, lemon juice, lime juice, mint syrup, rose syrup, berry compote, passion fruit syrup, mango puree, apricot jam, fig puree, date syrup, molasses, simple syrup, fruit juice, coconut cream


35. Invoke the CommaSeparatedListOutputParser to create a list of strings.

In [29]:
print(output_parser.invoke(response))

['pomegranate juice', 'cranberry juice', 'cherry juice', 'raspberry juice', 'strawberry syrup', 'elderflower cordial', 'orange juice', 'lemon juice', 'lime juice', 'mint syrup', 'rose syrup', 'berry compote', 'passion fruit syrup', 'mango puree', 'apricot jam', 'fig puree', 'date syrup', 'molasses', 'simple syrup', 'fruit juice', 'coconut cream']


**Task complete**: You made tools for the bar manager using LangChain components.

---

## Task 5: Create a chatbot

In this task, you will leverage the LangChain components you've already learned, to build a chatbot to help organize shifts for venue staff.

LangChain chat models are ideal for building chatbots

36. To relay its directive to the chatbot, define a system message.

In [30]:
system_message = SystemMessage(content="You are a chatbot built for scheduling staff shifts for AMusicVenue, an independent music venue.")

37. **Challenge**: Finish the shift booking chatbot that will invoke Mistral Large with a system message and a user input

    - Replace the `<FMI-1>` "Fill Me In (FMI)" value in the code with the appropriate message and its required parameter.

    **Tip:** Look for the message type to take a human message in [Langchain_core documentation](https://api.python.langchain.com/en/latest/core_api_reference.html#module-langchain_core.messages) and then find what parameter it takes.

In [31]:
nova_chat = ChatBedrock(model_id="amazon.nova-lite-v1:0")

try:
    # Print a welcome message to the user
    print("Welcome to this simple chatbot! To exit, choose the interrupt button and then press esc.")
    while True:
        # Prompt the user for input
        user_input = input("User: ")
        # Create a HumanMessage object with the user's input
        human_message = HumanMessage(content = user_input)
        # Print the user's input
        print(f"\nHuman: {user_input}")       
        print("AI: ", end="")
        
        # Stream the response from the chatbot
        for chunk in nova_chat.stream([system_message, human_message]):
            # Extract text from Nova's structured response format
            if hasattr(chunk, 'content') and chunk.content:
                if isinstance(chunk.content, list):
                    for item in chunk.content:
                        if isinstance(item, dict) and item.get('type') == 'text':
                            print(item.get('text', ''), end="", flush=True)
                else:
                    print(chunk.content, end="", flush=True)
        
        print("\n")  # Add newline after response

# Handle the KeyboardInterrupt exception (when the user presses esc)
except KeyboardInterrupt:
    pass


Welcome to this simple chatbot! To exit, choose the interrupt button and then press esc.

Human: This is Bob. I want to work Saturday.
AI: Hello Bob! I'd be happy to help you schedule your shift for Saturday at AMusicVenue. To get started, could you please provide me with the following information?

1. **Date and Time:** Which Saturday are you interested in working, and what specific hours would you prefer?
2. **Role:** Are you available to work as a bartender, stagehand, door staff, or another role?
3. **Availability:** Are there any other dates or times you are available or unavailable in the near future? This will help us create a balanced schedule.
4. **Preferences:** Do you have any particular preferences or constraints, such as not working consecutive weekends or needing time off for other commitments?

Once I have this information, I can check the availability and see if there's a spot for you on Saturday. Looking forward to your response!



<details>
    <summary><b>Solution:</b> Select if you need to see the solution.</summary>
    
<br/>

```python
human_message = HumanMessage(content = user_input)
```
</details>

38. Start the chatbot by running the above cell.

39. In the input pop-up at the top of the IDE, input the initial message "This is *your name*, I want to work Saturday."

40. Continue to respond to its questions, poking holes at its knowledge, until you see model is clearly confused.
- If it says it will check on availability, ask what it found.
- If the chatbot claims to schedule you for a shift, ask what position the shift is.

41. To exit, choose **Interrupt** from the main notebook editor toolbar, and then press the **Esc**/**esc** key.

**Warning:** If a pop up appears asking if "want to restart the kernel", choose **cancel** and then press the the **Esc**/**esc** key.

<img src="images/interrupt.png">

***Image description**: The *Interrupt* key within the IDE.

You likely had an interaction with the chatbot that hardly resembled a human interaction and was inadequate as a tool for the venue to use.

The chatbot has two primary flaws:
- It lacks statefullness, meaning it has no recollection of previous messages that you or it have delivered. It will have forgotten your name and what day you want to work by the time the next response is sent.
- It hallucinates, as it has no awarness of shift availability, so it tends to make information up.

In the next task, you will fix those two flaws.

**Task complete**: You created a chatbot for staffing the venue.

---

# Task 6: Make the chatbot application specific

In this task, you will add statefullness and context of needed shifts to the chatbot.

### Task 6.1: Make the chatbot stateful

Statelessness means LLMs do not retrain messages for their next invocation. Without the previous messages to the chatbot being stored, an LLMs utility is limited to a human message and an AI response.
So that venue staff can have a conversation with the chatbot, add statefullness to the chatbot.

42. **Challenge**: Finish adding statefullness to shift booking chatbot by 

    - Replace the `<FMI-1>` "Fill Me In (FMI)" value in the code with the appropriate method to add an object to the *messages* list.
    - Replace the `<FMI-2>` "Fill Me In (FMI)" value in the code with the appropriate parameter to add the *model response* to the messages list.

In [43]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a chatbot built for scheduling staff shifts for AMusicVenue, an independent music venue.")
]

try:
    # Print a welcome message to the user
    print("Welcome to this simple chatbot! To exit, choose the interrupt button and then press esc.")
    while True:
        # Prompt the user for input
        user_input = input("User: ")
        human_message = user_input
        print(f"Human:\n{user_input}\n---------------")
        # Create a HumanMessage object with the user's input and add it to the messages list
        messages.append(HumanMessage(human_message))

        # Assign a blank string that will be added to from the response chunks
        response_content = ""

        print("AI: ")
        # Stream the response from the chatbot and append each chunk to a string
        for chunk in nova_chat.stream(messages):
            # Extract text from Nova's structured response format
            if hasattr(chunk, 'content') and chunk.content:
                if isinstance(chunk.content, list):
                    for item in chunk.content:
                        if isinstance(item, dict) and item.get('type') == 'text':
                            text = item.get('text', '')
                            print(text, end="", flush=True)
                            response_content += text
                else:
                    print(chunk.content, end="", flush=True)
                    response_content += chunk.content
        
        # Add the AI message to the messages list
        messages.append(AIMessage(response_content))
        print(f"\n---------------")

# Handle the KeyboardInterrupt exception (when the user presses esc)
except KeyboardInterrupt:
    pass


Welcome to this simple chatbot! To exit, choose the interrupt button and then press esc.
Human:
i am a bob. I want to work Saterday.
---------------
AI: 
Hello Bob! I'd be happy to help you schedule your shift for Saturday. To get started, could you please provide me with the following information?

1. **Preferred Shift Time**: Are you available for the morning shift (e.g., 10 AM - 2 PM), afternoon shift (e.g., 2 PM - 6 PM), or evening shift (e.g., 6 PM - 10 PM)?
2. **Role**: Are you interested in working as a bartender, front-of-house staff, stage manager, or another role?
3. **Special Requirements**: Do you have any specific requests or requirements for your shift?

Once I have this information, I can check the availability and confirm your shift. Looking forward to your response!
---------------
Human:
i'm available for evening shift. The desired role - IT specialist. I have no special requirements.
---------------
AI: 
Great, Bob! Here's what we have for your Saturday shift:

**Dat

<details>
    <summary><b>Solution:</b> Select if you need to see the solution.</summary>
    
<br/>


```python
1. messages.append(HumanMessage(human_message))
```

```python
2. messages.append(AIMessage(response_content))
```
</details>

43. Start the chatbot by running the above cell.

44. In the input pop-up at the top of the IDE, input the initial message "This is *your name*, I want to work Saturday."

45. Continue to respond to its questions, until you see that the chatbot has gained statefullness.

46. To exit, choose **Interrupt** from the main notebook editor toolbar, and then press the **Esc**/**esc** key.

**Warning:** If a pop up appears asking if "want to restart the kernel", choose **cancel** and then press the the **Esc**/**esc** key.

<img src="images/interrupt.png">

***Image description**: The *Interrupt* key within the IDE.

Great! The chatbot has gained more function and is now stateful. But, it still lacks awareness of open shifts.

### Task 6.2: Give the chatbot context

On their own, LLMs are a fantastic tool. They are trained on mass amounts of data from the internet, making them incredibly knowledgeable. But, they don't know data specific to your use case. Add the functionality to pull data from a shift tracking CSV file to be able to effectively schedule shifts.

*Document loaders* are another way to add functionality to LangChain models. They load data from various sources, like AWS S3, JSON files, and URLs, into document objects. You can use those documents to provide context for model invocation.

There is a file in your directory called *shifts.csv*. It is a comma-seperated values file containing all remaining shifts the venue needs to staff for a given week.

47. To examine that file, select **shifts.csv** from the IDE sidebar.

48. To load the CSV file, with each line/available shift as its own document, run the following cell.

In [44]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="./shifts.csv")

documents = loader.load()

49. To make the documents usable for your purpose, iterate through them, splitting each *page_content* string and storing them in a list of shift dictionary objects.

In [45]:
# Initialize an empty list to store the shift dictionaries
shifts = []

# Iterate over each document in the loaded documents
for document in documents:
    # Get the page content of the current document as a string
    data_str = document.page_content
    # Create a dictionary from the string, splitting on newline ('\n') and colon (': ')
    # The keys of the dictionary are the parts before the colon, converted to lowercase
    # The values of the dictionary are the parts after the colon
    data_dict = {line.split(': ')[0].lower(): line.split(': ')[1] for line in data_str.split('\n')}
    # Add the newly created dictionary to the shifts list
    shifts.append(data_dict)

shifts

[{'time': '6PM-CLOSE',
  'day': 'Friday',
  'duty': 'Bar',
  'taken': 'None',
  'name': 'None',
  'phone': 'None'},
 {'time': '2PM-CLOSE',
  'day': 'Sunday',
  'duty': 'Front Door',
  'taken': 'Taken',
  'name': 'Mary Major',
  'phone': '2355550154'},
 {'time': '10AM-6PM',
  'day': 'Friday',
  'duty': 'Hospitality',
  'taken': 'None',
  'name': 'None',
  'phone': 'None'},
 {'time': '5PM-CLOSE',
  'day': 'Saturday',
  'duty': 'Bar',
  'taken': 'Taken',
  'name': 'Alejandro Rosalez',
  'phone': '2355550185'},
 {'time': '2PM-CLOSE',
  'day': 'Saturday',
  'duty': 'Front Door',
  'taken': 'None',
  'name': 'None',
  'phone': 'None'}]

50. To turn the list of dicts into a string usable by the chat model, run the following cell

In [46]:
# Initialize an empty string to store the shift information
shifts_string = ""
# Iterate over each shift dictionary in the shifts list
for shift in shifts:
    # If taken key does not equal taken, the shift is available
    if not(shift['taken'].lower() == 'taken'):
        # Ex: Front door needs staffing on Saturday, 5-CLOSE
        textrep = "{} needs staffing on {}, {}.".format(shift['duty'],shift['day'],shift['time'])
    # Shift is taken
    else:
        #Ex: Bar does not need staffing on Friday, 6-10, because Alejandro Rosalez is working it.
        textrep = "{} does not need staffing on {}, {}, because {} is working it.".format(shift['duty'],shift['day'],shift['time'],shift['name'])
    # Add the shift to the string of all shifts
    shifts_string += textrep + "\n"
print(shifts_string)

Bar needs staffing on Friday, 6PM-CLOSE.
Front Door does not need staffing on Sunday, 2PM-CLOSE, because Mary Major is working it.
Hospitality needs staffing on Friday, 10AM-6PM.
Bar does not need staffing on Saturday, 5PM-CLOSE, because Alejandro Rosalez is working it.
Front Door needs staffing on Saturday, 2PM-CLOSE.



51. **Challenge:** To give the chatbot the necessary context, add the string containing shifts, **shifts_string**, to the *SystemMessage*, and then start the chatbot by running the cell.

In [47]:
messages = [
    SystemMessage(content="You are a chatbot built for scheduling staff shifts for AMusicVenue, an independent music venue. You tell staff what shifts are available when they ask or that that they are not. These are the shifts this week:" + shifts_string)
]

try:
    # Print a welcome message to the user
    print("Welcome to this simple chatbot! To exit, choose the interrupt button and then press esc.")
    while True:
        # Prompt the user for input
        user_input = input("User: ")
        human_message = user_input
        print(f"Human:\n{user_input}\n---------------")
        # Create a HumanMessage object with the user's input and add it to the messages list
        messages.append(HumanMessage(human_message))

        # Assign a blank string that will be added to from the response chunks
        response_content = ""

        print("AI: ")
        # Stream the response from the chatbot and append each chunk to a string
        for chunk in nova_chat.stream(messages):
            # Extract text from Nova's structured response format
            if hasattr(chunk, 'content') and chunk.content:
                if isinstance(chunk.content, list):
                    for item in chunk.content:
                        if isinstance(item, dict) and item.get('type') == 'text':
                            text = item.get('text', '')
                            print(text, end="", flush=True)
                            response_content += text
                else:
                    print(chunk.content, end="", flush=True)
                    response_content += chunk.content
        ai_message = AIMessage(response_content)

        # Append the AI message to the messages list
        messages.append(ai_message)
        print(f"\n---------------")

# Handle the KeyboardInterrupt exception (when the user presses esc)
except KeyboardInterrupt:
    pass


Welcome to this simple chatbot! To exit, choose the interrupt button and then press esc.
Human:
i am Bob. I want to work Saturday.
---------------
AI: 
Hello Bob! Here are the shifts available for you on Saturday at AMusicVenue:

- **Front Door**: Staffing is needed from 2PM to CLOSE.
- **Bar**: Staffing is not needed from 5PM to CLOSE because Alejandro Rosalez is working it.

Would you like to sign up for the Front Door shift on Saturday?
---------------


<details>
    <summary><b>Solution:</b> Select if you need to see the solution.</summary>
    
<br/>

```python
messages = [
    SystemMessage(content="You are a chatbot built for scheduling staff shifts for AMusicVenue, an independent music venue. You tell staff what shifts are available when they ask or that that they are not. These are the shifts this week:" + shifts_string)
]
```

</details>

52. Input the initial message "This is *your name*, I want to work Saturday."

53. Continue to respond to its questions, seeing that the chatbot now has the context required to make intelligent staffing decisions.

54. To exit, choose **Interrupt** from the main notebook editor toolbar, and then press the **Esc**/**esc** key.

**Warning:** If a pop up appears asking if "want to restart the kernel", choose **cancel** and then press the the **Esc**/**esc** key.

<img src="images/interrupt.png">

***Image description**: The *Interrupt* key within the IDE.

LangChain has the tools to go even further: handling the logic, parsing, and secondary model invocation that would be required to update *shifts.csv* after a staff-member has agreed to take a shift, but you will not be learning those in this lab.

**Task complete**: You used LangChain to add statefullness and context to your staffing chatbot.

## Cleanup

You have completed this notebook. To continue to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab instructions and continue with the **Conclusion** section.